In [1]:
import torch

## Architecture of CNN

Image(1,28,28) -> Convolution layer(32 Filter / 3*3 / Zero Padding) </br>-> Max Pooling(2*2/ Stride = 2) -> Conv Layer(64 Filter/ 3*3/Zero Padding) -> Max Pooling(2*2/Stride = 2)</br> -> Flatten Layer -></br>Fully Connected layer with two hidden layer first one has 128 neurons and second one has 64 neurons and output layer has 10 neuron

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [3]:
torch.manual_seed(42)

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
device

'cpu'

In [7]:
df_train = pd.read_csv('fashion-mnist_train.csv')
df_test = pd.read_csv('fashion-mnist_test.csv')

In [ ]:
X_train = df_train.iloc[:,1:].values
y_train = df_train.iloc[:,0].values

In [9]:
X_test = df_test.iloc[:,1:].values
y_test = df_test.iloc[:,0].values

In [10]:

X_train = X_train/ 255.0
X_test = X_test/255.0

In [12]:
class CustomDataset(Dataset):
    def __init__(self,features,labels):
        self.features = torch.tensor(features, dtype=torch.float32).reshape(-1,1,28,28)
        self.labels = torch.tensor(labels, dtype=torch.long)
    def __len__(self):
        return len(self.features)
    def __getitem__(self,index):
        return self.features[index], self.labels[index]

In [13]:
train_dataset = CustomDataset(X_train, y_train)

In [14]:
test_dataset = CustomDataset(X_test, y_test)

In [25]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    pin_memory=False
)

In [26]:
test_dataloader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    pin_memory=False
)

In [27]:
class CNN(nn.Module):
    def __init__(self, input_features):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(input_features, 32, kernel_size=3, padding ='same'),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size = 2, stride=2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding ='same'),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size = 2, stride=2)
            
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*7*7,128),
            nn.ReLU(),
            nn.Dropout(p=0.4),
            
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Dropout(p=0.4),
            nn.Linear(64,10)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        
        return x

In [19]:
LEARNING_RATE = 0.01
EPOCHS = 100

In [29]:
model = CNN(1)
model.to(device)

CNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): ReLU()
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.4, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.4, inplace=False)
    (7): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [31]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)

In [32]:
#training loop

for epoch in range(EPOCHS):
    total_epoch_loss = 0
    for batch_features, batch_labels in train_dataloader:
        #forward pass
        outputs = model(batch_features)
        
        
        #loss calculate
        loss = loss_fn(outputs, batch_labels)
        
        #back pass
        optimizer.zero_grad()
        loss.backward()
        
        #update grads
        optimizer.step()
        
        total_epoch_loss = total_epoch_loss + loss.item()
    
    avg_loss = total_epoch_loss / len(train_dataloader)
    print(f"Epoch: {epoch + 1}, Loss: {avg_loss}")
    

Epoch: 1, Loss: 0.5565372070590655
Epoch: 2, Loss: 0.38896417917609216
Epoch: 3, Loss: 0.3294265355706215
Epoch: 4, Loss: 0.29496507491767404
Epoch: 5, Loss: 0.2671063875953356
Epoch: 6, Loss: 0.24907775012155373
Epoch: 7, Loss: 0.23399355927457413
Epoch: 8, Loss: 0.2182099217514197
Epoch: 9, Loss: 0.20954758679469426
Epoch: 10, Loss: 0.2018285897642374
Epoch: 11, Loss: 0.19239087854574125
Epoch: 12, Loss: 0.1886153807774186
Epoch: 13, Loss: 0.18343479975797236
Epoch: 14, Loss: 0.17582301047394672
Epoch: 15, Loss: 0.17795019639780124
Epoch: 16, Loss: 0.16325064144780238
Epoch: 17, Loss: 0.15906339834108948
Epoch: 18, Loss: 0.15689037860191116
Epoch: 19, Loss: 0.14701180656142532
Epoch: 20, Loss: 0.14281285925103973
Epoch: 21, Loss: 0.1463227993450438
Epoch: 22, Loss: 0.1405471598888437
Epoch: 23, Loss: 0.13602977875762928
Epoch: 24, Loss: 0.1307904679781447
Epoch: 25, Loss: 0.1280647086670622
Epoch: 26, Loss: 0.12819624319672585
Epoch: 27, Loss: 0.12696079261476795
Epoch: 28, Loss: 0.1

In [33]:
#evaluation code

total = 0
correct = 0


with torch.no_grad():
    for batch_features, batch_labels in test_dataloader:
        
        outputs = model(batch_features) # shape of outputs will be (32,10) like each data will have 10 prob.
        predicted = outputs.argmax(dim=1)
        
    total = total + batch_labels.shape[0]
    correct = correct + (predicted == batch_labels).sum().item()
print(f"Testing accuarcy is {correct*100/total}" )
        

Testing accuarcy is 93.75


In [35]:

#evaluation code

total = 0
correct = 0


with torch.no_grad():
    for batch_features, batch_labels in train_dataloader:
        
        outputs = model(batch_features) # shape of outputs will be (32,10) like each data will have 10 prob.
        predicted = outputs.argmax(dim=1)
        
    total = total + batch_labels.shape[0]
    correct = correct + (predicted == batch_labels).sum().item()
print(f"Training accuracy is {correct*100/total}" )
        

Training accuracy is 96.875
